## Display & Joystick의 객체화 및 캐릭터 조종

필요한 library import

In [65]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw
from adafruit_rgb_display import st7789

Joystick이라는 클래스를 만들어, 간단한 모듈화 진행

In [66]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height

joystick 객체 선언

In [67]:
joystick = Joystick()

In [68]:
map=Image.open("./res/map.png").resize((240,240))
character=Image.open("./res/ch_1_right.png").resize((25,25))
map.paste(character,(40,40,65,65),character)
joystick.disp.image(map)

## 캐릭터 만들기

간단한 모양(네모, 동그라미, ..)을 가지고 조종할 수 있는 캐릭터를 만들어보기
- 기능 -> 움직이기, 움직일때는 빨간 테두리를, 가만히 있을 때는 하얀 테두리를 가지게 할 것

In [69]:
import numpy as np
class Character:
    def __init__(self):
        self.appearance = "character"
        self.state = None
        self.position = [15, 210, 40, 235]
        self.size = (25,25)
        self.image = Image.open("./res/ch_1_right.png").resize(self.size)
        self.flag = 0
        self.count = 0
        self.direction = "right"
        self.gravity=9

    def move(self, command):
    #     if command == "up":
    #         self.position[1] -= 5
    #         self.position[3] -= 5

    #     elif command == "down":
    #         self.position[1] += 5
    #         self.position[3] += 5

        if command == "left":
            self.direction = "left"
            self.flag = 0 if self.flag else 1
            if self.flag:
                self.image = Image.open("./res/ch_1_left.png").resize(self.size)
            else:
                self.image = Image.open("./res/ch_2_left.png").resize(self.size)
            self.position[0] -= 5
            self.position[2] -= 5
                
        elif command == "right":
            self.direction = "right"
            self.flag = 0 if self.flag else 1
            if self.flag:
                self.image = Image.open("./res/ch_1_right.png").resize(self.size)
            else:
                self.image = Image.open("./res/ch_2_right.png").resize(self.size)
            self.position[0] += 5
            self.position[2] += 5

    def jump(self):
        if self.count < 8:
            if self.direction=="right":
                self.image = Image.open("./res/ch_r_j.png").resize(self.size)
            else:
                self.image = Image.open("./res/ch_l_j.png").resize(self.size)
            
            self.position[1] -= self.gravity
            self.position[3] -= self.gravity
            self.gravity -= 1
            self.count += 1


        elif self.count < 16:
            if self.direction=="right":
                self.image = Image.open("./res/ch_r_d.png").resize(self.size)
            else:
                self.image = Image.open("./res/ch_l_d.png").resize(self.size)
            
            self.gravity += 1
            self.position[1] += self.gravity
            self.position[3] += self.gravity
            self.count+=1

        else:
            if self.direction=="right":
                self.image = Image.open("./res/ch_1_right.png").resize(self.size)
            else:
                self.image = Image.open("./res/ch_1_left.png").resize(self.size)
            self.count=0
            self.state = None
        
    def attack(self):
        if self.direction=="right":
                self.image = Image.open("./res/ch_a_r.png").resize(self.size)
        else:
            self.image = Image.open("./res/ch_a_l.png").resize(self.size)

        
        







In [70]:
# 잔상이 남지 않는 코드
character = Character()

map = Image.open("./res/map2.png").resize((240,240))
while True:
    my_map=map.copy()
    # if not joystick.button_U.value:  # up pressed
    #     character.move('up')

    # if not joystick.button_D.value:  # down pressed
    #     character.move('down')

    if not joystick.button_L.value:  # left pressed
        character.move('left')

    if not joystick.button_R.value:  # right pressed
        character.move('right')

    if not joystick.button_B.value:
        character.state = 'jump'

    if character.state == 'jump':
        character.jump()
        
    if not joystick.button_A.value:  # right pressed
        character.attack()
        

    
    my_map.paste(character.image, character.position, character.image)
    joystick.disp.image(my_map)

KeyboardInterrupt: 